# Classification codes:

In [ ]:
import numpy as np
from random import randint
from math import floor, fabs
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys
import time
import pandas as pd
from itertools import product
import random
from sklearn import preprocessing
import copy
import itertools
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.datasets import make_gaussian_quantiles
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression

In [ ]:
examples=['data/NewExample1','data/NewExample1Redundant0.1','data/NewExample1Redundant0.25','data/NewExample1Redundant0.5']

ZeroProb=[0.0]
           
for example in examples:
    for Z in ZeroProb:
        dfTrain = pd.read_csv(example+'train.csv', delimiter=",")
        del dfTrain['Unnamed: 0']
        dfTest=pd.read_csv(example+'test.csv', delimiter=",")
        del dfTest['Unnamed: 0']     
        X_trainset = dfTrain.iloc[:, :-1].values
        y_trainset = dfTrain.iloc[:, -1].values
        X_testset = dfTest.iloc[:, :-1].values
        y_testset = dfTest.iloc[:, -1].values

        ###########random forest
        estimators=[18]
        with open('randomforestResults.txt', 'a') as f:
            f.write('{},{}\n'.format('########################',example))
        f.closed            

        for n in estimators:    
            clf = RandomForestClassifier( n_estimators = n)
            TrainStartTime=time.time()
            clf.fit(X_trainset,y_trainset)
            fitTime=time.time()-TrainStartTime
            predclf = clf.predict(X_trainset)
            with open('randomforestResults.txt', 'a') as f:
                f.write('{},{}\n'.format('************random forest**********',n))
                f.write('{}:{},{}:{},{}:{}\n'.format('Train fitTime',fitTime,'Train Accuracy',metrics.accuracy_score(y_trainset, predclf),'Train Confusion Matrix',confusion_matrix(y_trainset, predclf, labels=[0,1])))
            f.closed
            TestStartTime=time.time()
            predclf = clf.predict(X_testset)
            PredictTime=time.time()-TestStartTime
            with open('randomforestResults.txt', 'a') as f:
                f.write('{}:{},{}:{},{}:{}\n'.format('Test fitTime',PredictTime/len(dfTest),'Test Accuracy',metrics.accuracy_score(y_testset, predclf),'Test Confusion Matrix',confusion_matrix(y_testset, predclf, labels=[0,1])))
            f.closed               
        ########################### knn
        KK=[4]
        with open('knnResults.txt', 'a') as f:
            f.write('{},{}\n'.format('########################',example))
        f.closed            


        for k in KK:
            TrainStartTime=time.time()
            neigh = KNeighborsClassifier(n_neighbors = k).fit(X_trainset,y_trainset)
            fitTime=time.time()-TrainStartTime
            PredKnn=neigh.predict(X_trainset)
            with open('knnResults.txt', 'a') as f:
                f.write('{},{}\n'.format('************knn**********',k))
                f.write('{}:{},{}:{},{}:{}\n'.format('Train fitTime',fitTime,'Train Accuracy',metrics.accuracy_score(y_trainset, PredKnn),'Train Confusion Matrix',confusion_matrix(y_trainset, PredKnn, labels=[0,1])))
            f.closed

            TestStartTime=time.time()
            PredKnn=neigh.predict(X_testset)
            PredictTime=time.time()-TestStartTime               
            with open('knnResults.txt', 'a') as f:
                f.write('{}:{},{}:{},{}:{}\n'.format('Test fitTime',PredictTime/len(dfTest),'Test Accuracy',metrics.accuracy_score(y_testset, PredKnn),'Test Confusion Matrix',confusion_matrix(y_testset, PredKnn, labels=[0,1])))
            f.closed 

        KK=[4]
        with open('LogisticRegressionResults.txt', 'a') as f:
            f.write('{},{}\n'.format('########################',example))
        f.closed                      
        sol=["liblinear"]
        c=[1]
        Prob=[0.5]
        for i in range(len(sol)):
            for n in (c):
                for t in range(len(Prob)):
                    TrainStartTime=time.time()
                    LR = LogisticRegression(C=n, solver=sol[i]).fit(X_trainset,y_trainset)
                    fitTime=time.time()-TrainStartTime
                    LRpredict  = (LR.predict_proba(X_trainset)[:, 1] >= Prob[t]).astype(int) 
                    with open('LogisticRegressionResults.txt', 'a') as f:
                        f.write('{},{},{}\n'.format('************Logistic Regression**********',sol[i],n))
                        f.write('{}:{},{}:{},{}:{}\n'.format('Train fitTime',fitTime,'Train Accuracy',metrics.accuracy_score(y_trainset, LRpredict),'Train Confusion Matrix',confusion_matrix(y_trainset, LRpredict, labels=[0,1])))
                    f.closed
                    TestStartTime=time.time()
                    LRpredict  = (LR.predict_proba(X_testset)[:, 1] >=  Prob[t]).astype(int) 
                    PredictTime=time.time()-TestStartTime
                    with open('LogisticRegressionResults.txt', 'a') as f:
                        f.write('{}:{},{}:{},{}:{}\n'.format('Test fitTime',PredictTime/len(dfTest),'Test Accuracy',metrics.accuracy_score(y_testset, LRpredict),'Test Confusion Matrix',confusion_matrix(y_testset, LRpredict, labels=[0,1])))
                    f.closed 


        #################for train SVM
        with open('SVMResults.txt', 'a') as f:
            f.write('{},{}\n'.format('########################',example))
        f.closed             
        kern=["linear"]
        c=[0.4]
        for t in range(len(kern)):
            for i in range(len(c)):
                TrainStartTime=time.time()    
                clf = svm.SVC(kernel=kern[t],C=c[i])
                clf.fit(X_trainset, y_trainset) 
                fitTime=time.time()-TrainStartTime        
                SVMpredict = clf.predict(X_trainset)
                with open('SVMResults.txt', 'a') as f:
                    f.write('{},{},{}\n'.format('************SVM**********',kern[t],c[i]))
                    f.write('{}:{},{}:{},{}:{}\n'.format('Train fitTime',fitTime,'Train Accuracy',metrics.accuracy_score(y_trainset, SVMpredict),'Train Confusion Matrix',confusion_matrix(y_trainset, SVMpredict, labels=[0,1])))
                f.closed                    
                TestStartTime=time.time()
                SVMpredict = clf.predict(X_testset)
                PredictTime=time.time()-TestStartTime
                with open('SVMResults.txt', 'a') as f:
                    f.write('{}:{},{}:{},{}:{}\n'.format('Test fitTime',PredictTime/len(dfTest),'Test Accuracy',metrics.accuracy_score(y_testset, SVMpredict),'Test Confusion Matrix',confusion_matrix(y_testset, SVMpredict, labels=[0,1])))
                f.closed  


        #################for train Neural Network    
        HiddenSize=[6]
        Active=['identity']
        Solve=[ 'sgd']
        with open('NeuralNetworkResults.txt', 'a') as f:
            f.write('{}\n'.format('########################',example))
        f.closed     
        for i in HiddenSize:
            for act in range(len(Active)):
                for s in range(len(Solve)):
                    NN1=MLPClassifier(hidden_layer_sizes=i,activation=Active[act],solver=Solve[s],alpha=0.01,
                                      batch_size=100,learning_rate='adaptive',
                                      learning_rate_init=0.0007,max_iter=1500,verbose=True) 
                    TrainStartTime=time.time() 
                    NN1.fit(X_trainset,y_trainset)                        
                    fitTime=time.time()-TrainStartTime                              
                    Yhat1_tr=NN1.predict(X_trainset)  # make prediction for training data

                    with open('NeuralNetworkResults.txt', 'a') as f:
                        f.write('{},{},{},{}\n'.format('************Neural Net**********',i,Active[act],Solve[s]))
                        f.write('{}:{},{}:{},{}:{}\n'.format('Train fitTime',fitTime,'Train Accuracy',metrics.accuracy_score(y_trainset, Yhat1_tr),'Train Confusion Matrix',confusion_matrix(y_trainset, Yhat1_tr, labels=[0,1])))
                    f.closed 
                    TestStartTime=time.time()
                    Yhat1_te=NN1.predict(X_testset) 
                    PredictTime=time.time()-TestStartTime
                    with open('NeuralNetworkResults.txt', 'a') as f:
                        f.write('{}:{},{}:{},{}:{}\n'.format('Test fitTime',PredictTime/len(dfTest),'Test Accuracy',metrics.accuracy_score(y_testset, Yhat1_te),'Test Confusion Matrix',confusion_matrix(y_testset, Yhat1_te, labels=[0,1])))
                    f.closed  
